# Creating relations in SQLite

In [1]:
import sqlite3
conn = sqlite3.connect('nominations.db')

In [2]:
schema=conn.execute('pragma table_info(nominations)')
first_ten=conn.execute('select * from nominations limit 10')
for item in schema:
    print(item)
for item in first_ten:
    print(item)

(0, u'Year', u'INTEGER', 0, None, 0)
(1, u'Category', u'TEXT', 0, None, 0)
(2, u'Nominee', u'TEXT', 0, None, 0)
(3, u'Won', u'INTEGER', 0, None, 0)
(4, u'Movie', u'TEXT', 0, None, 0)
(5, u'Character', u'TEXT', 0, None, 0)
(2010, u'Actor -- Leading Role', u'Javier Bardem', 0, u'Biutiful ', u'Uxbal')
(2010, u'Actor -- Leading Role', u'Jeff Bridges', 0, u'True Grit ', u'Rooster Cogburn')
(2010, u'Actor -- Leading Role', u'Jesse Eisenberg', 0, u'The Social Network ', u'Mark Zuckerberg')
(2010, u'Actor -- Leading Role', u'Colin Firth', 1, u"The King's Speech ", u'King George VI')
(2010, u'Actor -- Leading Role', u'James Franco', 0, u'127 Hours ', u'Aron Ralston')
(2010, u'Actor -- Supporting Role', u'Christian Bale', 1, u'The Fighter ', u'Dicky Eklund')
(2010, u'Actor -- Supporting Role', u'John Hawkes', 0, u"Winter's Bone ", u'Teardrop')
(2010, u'Actor -- Supporting Role', u'Jeremy Renner', 0, u'The Town ', u'James Coughlin')
(2010, u'Actor -- Supporting Role', u'Mark Ruffalo', 0, u'The Ki

## Create table for normalization

### Create table

In [3]:
q='''create table ceremonies(
id integer primary key,
year integer,
host text
)'''
conn.execute(q)

### Usng .executemany

In [4]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal")
            ]

insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"
conn.executemany(insert_query, years_hosts)

In [5]:
print(conn.execute('select * from ceremonies limit 10').fetchall())
print(conn.execute('pragma table_info(ceremonies)').fetchall())

[(1, 2010, u'Steve Martin'), (2, 2009, u'Hugh Jackman'), (3, 2008, u'Jon Stewart'), (4, 2007, u'Ellen DeGeneres'), (5, 2006, u'Jon Stewart'), (6, 2005, u'Chris Rock'), (7, 2004, u'Billy Crystal'), (8, 2003, u'Steve Martin'), (9, 2002, u'Whoopi Goldberg'), (10, 2001, u'Steve Martin')]
[(0, u'id', u'integer', 0, None, 1), (1, u'year', u'integer', 0, None, 0), (2, u'host', u'text', 0, None, 0)]


## Foreign key constraints
makes sure that SQLite makes sure that the foreign key value exists

**Needs to be executed for EVERY SQLite connection**

In [6]:
conn.execute('pragma foreign_keys=on;')

## One-to-many setup
SQLite cannot delete columns, so in order to set up a new table:
- create new table 'nominations_two'
- populate nominations_two with desired data
- delete original nominations table
- rename nominations_two to nominations

In [7]:
create_nominations_two = '''create table nominations_two 
(id integer primary key, 
category text, 
nominee text, 
movie text, 
character text, 
won text,
ceremony_id integer,
foreign key(ceremony_id) references ceremonies(id));
'''

nom_query = '''
select ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie as movie, 
nominations.character as character, nominations.won as won
from nominations
inner join ceremonies 
on nominations.year == ceremonies.year
;
'''
# join the two tables
joined_nominations = conn.execute(nom_query).fetchall()
# create empty table nominations_two
conn.execute(create_nominations_two)

# insert into empty table all the values of joined_nominations
insert_nominations_two = '''insert into nominations_two (ceremony_id, category, nominee, movie, character, won) 
values (?,?,?,?,?,?);
'''
conn.executemany(insert_nominations_two, joined_nominations)

print(conn.execute("select * from nominations_two limit 5;").fetchall())

[(1, u'Actor -- Leading Role', u'Javier Bardem', u'Biutiful ', u'Uxbal', u'0', 1), (2, u'Actor -- Leading Role', u'Jeff Bridges', u'True Grit ', u'Rooster Cogburn', u'0', 1), (3, u'Actor -- Leading Role', u'Jesse Eisenberg', u'The Social Network ', u'Mark Zuckerberg', u'0', 1), (4, u'Actor -- Leading Role', u'Colin Firth', u"The King's Speech ", u'King George VI', u'1', 1), (5, u'Actor -- Leading Role', u'James Franco', u'127 Hours ', u'Aron Ralston', u'0', 1)]


## Delete, Rename tables
### Deleting old table
- drop table : deletes table

In [8]:
conn.execute('drop table nominations')

### Rename table
- alter 'table1' rename to 'table2' : changes the name of table1 to table2

In [9]:
conn.execute('alter table nominations_two rename to nominations')

Create the 3 tables we need to model the relationship between movies and actors. You need to create the movies and actors tables before creating the movies_actors table for the foreign key references to work.
Create the movies table using the following schema:
id: primary key, integer type.
movie: movie name, text type.
Create the actors table using the following schema:
id: primary key, integer type.
actor: actor's full name, text type.
Create the movies_actors join table using the following schema:
id: primary key, integer type.
movie_id: foreign key reference to movies.id column.
actor_id: foreign key reference to actors.id column.

## Many-to-many setup

In [14]:
# create movies table
q_movies='''create table movies( id integer primary key, movie text)'''
q_actors='''create table actors( id integer primary key, actor text)'''
q_movies_actors='''create table movies_actors(id integer primary key, movie_id integer references movies(id),
actor_id integer references actors(id))   '''
conn.execute(q_movies)
conn.execute(q_actors)
conn.execute(q_movies_actors)